In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [114]:
# Import
import gensim.downloader

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

from sklearn import preprocessing
from sklearn.pipeline import make_pipeline

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split

from statistics import mean

import numpy as np
import pandas as pd

import _helpers as hp

# Load Data

In [115]:
ASR_data = pd.read_csv(
    "snips/new_ASR_with_labels.csv"
)  # ASR data with improved speech recognition with 15555 framerate and with autocorrection applied

GT_data = pd.read_csv("snips/merged_GT_data.csv")  # Groundtruth data

## Data manipulation

In [116]:
# Get ASR data into a numpy word array per sentence plus a numpy label array

XX_ASR = ASR_data["transcript"].apply(lambda x: x.split(" "))
X_ASR = list(XX_ASR)  # numpy word array per transcript

y_num_ASR = np.array(ASR_data["user_action_num"])  # labels

In [117]:
# Get Groundtruth data into a numpy word array per sentence plus a numpy label array

XX_GT = GT_data["transcript"].apply(lambda x: x.split(" "))
X_GT = list(XX_GT)  # numpy word array per transcript

y_num_GT = np.array(GT_data["user_action_num"])  # labels

# Word2Vec

## Import pre-trained Word2Vec model

In [118]:
model_w2v = gensim.downloader.load("glove-wiki-gigaword-100")

## Get features and labels

In [119]:
def get_Word2Vec_feature(sentence):
    """return word2vec numpy array representation of sentence"""

    value_iter = np.zeros((model_w2v.vector_size,))

    for word in sentence:
        try:
            word_vec = model_w2v[word]
            value_iter += np.array(word_vec)
        except:
            continue

    return value_iter

In [120]:
# features Space
features_ASR = np.stack(XX_ASR.apply(get_Word2Vec_feature).values, axis=0)

# features Space
features_GT = np.stack(XX_GT.apply(get_Word2Vec_feature).values, axis=0)

In [121]:
labels_ASR = ASR_data["user_action_num"]
labels_GT = GT_data["user_action_num"]

# Classifiers

## Train/Test Split

The function beneath provides the features and labels needed for testing. Using the loaded ASR or not (then using ground truth data) is decided by input. As standard we use the ASR dataset.

In [122]:
def get_train_test_data(type_of_dataset="ASR", train_size=0.9):
    """Retrieves the relevant dataset and splits according to parameter"""
    # If ASR, give ASR features and labels
    if type_of_dataset == "ASR":
        train_features, test_features, train_labels, test_labels = train_test_split(
            features_ASR, labels_ASR, train_size=train_size
        )
    # If the dataset is not the ASR data, use the ground truth data
    else:
        train_features, test_features, train_labels, test_labels = train_test_split(
            features_GT, labels_GT, train_size=train_size
        )

    return train_features, test_features, train_labels, test_labels


def run_classifier(
    classifier_pipe, type_of_dataset="ASR", train_size=0.9, number_of_times=100
):
    """For running the classifiers multiple times, and returning mean accuracy score. Wraps around get_train_test_data"""
    mean_score_list = []
    n = number_of_times
    for i in range(n):
        train_features, test_features, train_labels, test_labels = get_train_test_data(
            type_of_dataset="ASR"
        )
        classifier_pipe.fit(train_features, train_labels)

        classifier_pred_labels = classifier_pipe.predict(test_features)  # predictions

        classifier_score = classifier_pipe.score(test_features, test_labels)  # accuracy

        mean_score_list.append(lgr_score)
    return mean_score, classifier_pred_labels, classifier_score

## Logistic Regression

In [123]:
lgr = LogisticRegression(C=0.06, max_iter=1000)  # Create the classification model

lgr_pipe = make_pipeline(preprocessing.StandardScaler(), lgr)  # Scale feature space

mean_score, lgr_pred_labels, lgr_score = run_classifier(
    classifier_pipe=lgr_pipe,
    type_of_dataset="ASR",
    train_size=0.9,
    number_of_times=100,
)

print("Average accuracy score =", round(mean_score, 3))

Average accuracy score = 0.782


### Model Evaluation
Gives detail on the last run of the classifier

In [124]:
print(
    classification_report(
        test_labels,
        lgr_pred_labels,
        target_names=[
            "SwitchLightOff",
            "SwitchLightOn",
            "IncreaseBrightness",
            "DecreaseBrightness",
            "SetLightBrightness",
            "SetLightColor",
        ],
    )
)

print(confusion_matrix(test_labels, lgr_pred_labels))

print("\nACCURACY:", lgr_score)

                    precision    recall  f1-score   support

    SwitchLightOff       0.11      0.10      0.11        29
     SwitchLightOn       0.30      0.24      0.27        33
IncreaseBrightness       0.12      0.11      0.11        28
DecreaseBrightness       0.12      0.12      0.12        24
SetLightBrightness       0.22      0.25      0.23        28
     SetLightColor       0.07      0.08      0.08        24

          accuracy                           0.16       166
         macro avg       0.16      0.15      0.15       166
      weighted avg       0.16      0.16      0.16       166

[[3 8 3 5 5 5]
 [3 8 7 5 6 4]
 [4 2 3 5 8 6]
 [4 2 2 3 5 8]
 [8 3 4 2 7 4]
 [5 4 6 6 1 2]]

ACCURACY: 0.8313253012048193


# Naive Bayes

### Gaussian Naive Bayes

In [125]:
gnb = GaussianNB()  # Create the classification model

gnb_pipe = make_pipeline(preprocessing.StandardScaler(), gnb)  # Scale feature space

mean_score, gnb_pred_labels, gnb_score = run_classifier(
    classifier_pipe=gnb_pipe,
    type_of_dataset="ASR",
    train_size=0.9,
    number_of_times=100,
)

print("Average accuracy score =", round(mean_score, 3))

Average accuracy score = 0.782


#### Model Evaluation

In [126]:
print(
    classification_report(
        test_labels,
        gnb_pred_labels,
        target_names=[
            "SwitchLightOff",
            "SwitchLightOn",
            "IncreaseBrightness",
            "DecreaseBrightness",
            "SetLightBrightness",
            "SetLightColor",
        ],
    )
)

print(confusion_matrix(test_labels, gnb_pred_labels))

print("\nACCURACY:", gnb_score)

                    precision    recall  f1-score   support

    SwitchLightOff       0.10      0.07      0.08        29
     SwitchLightOn       0.20      0.24      0.22        33
IncreaseBrightness       0.20      0.18      0.19        28
DecreaseBrightness       0.05      0.04      0.04        24
SetLightBrightness       0.17      0.25      0.20        28
     SetLightColor       0.39      0.29      0.33        24

          accuracy                           0.18       166
         macro avg       0.18      0.18      0.18       166
      weighted avg       0.18      0.18      0.18       166

[[2 9 6 2 7 3]
 [4 8 4 6 9 2]
 [8 6 5 1 7 1]
 [4 7 3 1 8 1]
 [0 6 5 6 7 4]
 [2 5 2 5 3 7]]

ACCURACY: 0.4759036144578313


# SVM

In [127]:
svm = SVC()  # Create the classification model

svm_pipe = make_pipeline(preprocessing.Normalizer(), svm)  # Scale feature space

mean_score, svm_pred_labels, svm_score = run_classifier(
    classifier_pipe=svm_pipe,
    type_of_dataset="ASR",
    train_size=0.9,
    number_of_times=100,
)

print("Average accuracy score =", round(mean_score, 3))

Average accuracy score = 0.782


### Model Evaluation

In [128]:
print(
    classification_report(
        test_labels,
        svm_pred_labels,
        target_names=[
            "SwitchLightOff",
            "SwitchLightOn",
            "IncreaseBrightness",
            "DecreaseBrightness",
            "SetLightBrightness",
            "SetLightColor",
        ],
    )
)

print(confusion_matrix(test_labels, svm_pred_labels))

print("\nACCURACY:", svm_score)

                    precision    recall  f1-score   support

    SwitchLightOff       0.22      0.24      0.23        29
     SwitchLightOn       0.20      0.21      0.21        33
IncreaseBrightness       0.11      0.11      0.11        28
DecreaseBrightness       0.05      0.04      0.05        24
SetLightBrightness       0.08      0.07      0.08        28
     SetLightColor       0.19      0.21      0.20        24

          accuracy                           0.15       166
         macro avg       0.14      0.15      0.14       166
      weighted avg       0.14      0.15      0.15       166

[[7 5 4 2 6 5]
 [7 7 7 5 4 3]
 [6 5 3 6 3 5]
 [4 4 5 1 7 3]
 [4 7 6 4 2 5]
 [4 7 3 2 3 5]]

ACCURACY: 0.7590361445783133


# NEURAL NETWORK

In [129]:
mlp = MLPClassifier(
    hidden_layer_sizes=(100, 100, 100), activation="relu", solver="adam", max_iter=5000
)  # Create the classification model

mlp_pipe = make_pipeline(preprocessing.Normalizer(), mlp)  # Scale feature space

mean_score, mlp_pred_labels, mlp_score = run_classifier(
    classifier_pipe=mlp_pipe,
    type_of_dataset="ASR",
    train_size=0.9,
    number_of_times=100,
)

print("Average accuracy score =", round(mean_score, 3))

Average accuracy score = 0.782


### Model Evaluation

In [130]:
print(
    classification_report(
        test_labels,
        mlp_pred_labels,
        target_names=[
            "SwitchLightOff",
            "SwitchLightOn",
            "IncreaseBrightness",
            "DecreaseBrightness",
            "SetLightBrightness",
            "SetLightColor",
        ],
    )
)

print(confusion_matrix(test_labels, mlp_pred_labels))

print("\nACCURACY:", mlp_score)

                    precision    recall  f1-score   support

    SwitchLightOff       0.18      0.17      0.18        29
     SwitchLightOn       0.06      0.03      0.04        33
IncreaseBrightness       0.26      0.36      0.30        28
DecreaseBrightness       0.14      0.08      0.11        24
SetLightBrightness       0.02      0.04      0.03        28
     SetLightColor       0.15      0.17      0.16        24

          accuracy                           0.14       166
         macro avg       0.14      0.14      0.13       166
      weighted avg       0.13      0.14      0.13       166

[[ 5  4  6  2  5  7]
 [ 5  1  6  3 12  6]
 [ 3  1 10  1  8  5]
 [ 4  3  5  2  9  1]
 [ 6  7  7  3  1  4]
 [ 5  0  4  3  8  4]]

ACCURACY: 0.8433734939759037


# Try Your Self

In [131]:
def user_friendly(sentence, cls):
    """return action from sentence"""

    sent = sentence.split()
    new_sent = []

    for word in sent:
        new_sent.append(hp.autocorrection(word))

    x_pred = get_Word2Vec_feature(new_sent).reshape(1, -1)
    y_pred = cls.predict(x_pred)
    return hp.indx2action(y_pred)

In [132]:
user_friendly("I want to set brithenss to fifty in my living room", lgr_pipe)

['SetLightBrightness']